<a href="https://colab.research.google.com/github/NamanMohindra/EnronClassifier-/blob/master/Context_Classification(8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import re
import os
import numpy as np
import random
from numpy import dot
from numpy.linalg import norm


In [3]:
ENRONFOLDER = 'drive/My Drive/Enron Spam/'
ENRONNOSPAMDATASET = 'drive/My Drive/Enron Spam/ham'
FEATURES_LOC = 'drive/My Drive/Enron Spam/featureVectors.npy'

feature_vectors = np.load(FEATURES_LOC)

#1) Affirmative Context:
#(a) Thank You, I will look into it
#(b) Thank you for informing
#
#2) Proposal Context:
#(a) Sure, we can proceed
#(b) I am not sure we can proceed
# 
#3) Follow up Context:
#(a) Okay, I will follow up as required
#(b) Unfortunately, I am unable to follow up
#
#Invite context:
#(a) Thank You, I will be there
#(b) Thank you, but I won't be able to make it

#Each context has a flag and an associated context vector(centroid) (just like formal/informal classification)

affirmative_flag = 0
proposal_flag = 0
followup_flag = 0
invite_flag = 0

affirmative_centroid = np.zeros(1000)
proposal_centroid = np.zeros(1000)
followup_centroid = np.zeros(1000)
invite_centroid = np.zeros(1000)

learning_rate = 0.1

annotation_indices = []
annotations = []

print("Feature Vectors loaded! Dimensions of the feature vectors matrix: " + str(np.shape(feature_vectors)))

Feature Vectors loaded! Dimensions of the feature vectors matrix: (3691, 1000)


In [0]:
def getLabel(mailIndex):
  file_location = os.listdir(ENRONNOSPAMDATASET)[mailIndex]
  FILENAME = os.path.join(ENRONNOSPAMDATASET, file_location)
  with open(FILENAME, 'r',encoding='utf-8',errors='ignore') as myfile:
    data = myfile.read()
    print("Message Contents: ")
    print(data)
    print("\n Is the context of the displayed displayed message affirmative[0] or proposal[1] or followup[2] or invite[3] ?")
    val = int(input("Enter your value: "))
    return val

def centroid_update(anno_index, label_value):
  
  global affirmative_flag
  global proposal_flag
  global followup_flag
  global invite_flag

  global affirmative_centroid
  global proposal_centroid
  global followup_centroid
  global invite_centroid

  global learning_rate

  fvector = feature_vectors[anno_index]

  a_flag = affirmative_flag
  p_flag = proposal_flag
  f_flag = followup_flag
  i_flag = invite_flag

  if(label_value==0):
    if(a_flag == 0):
      affirmative_flag = 1
      affirmative_centroid = fvector
    else:
      diff = np.subtract(fvector, affirmative_centroid)
      mult = np.multiply(diff, learning_rate)
      affirmative_centroid = np.add(mult, affirmative_centroid)
    if(p_flag == 1):
      diff = np.subtract(fvector, proposal_centroid)
      mult = np.multiply(diff, learning_rate)
      proposal_centroid = np.subtract(proposal_centroid, mult)
    if(f_flag == 1):
      diff = np.subtract(fvector, followup_centroid)
      mult = np.multiply(diff, learning_rate)
      followup_centroid = np.subtract(followup_centroid, mult)
    if(i_flag == 1):
      diff = np.subtract(fvector, invite_centroid)
      mult = np.multiply(diff, learning_rate)
      invite_centroid = np.subtract(invite_centroid, mult)
    
  
  elif(label_value==1):
    if(p_flag == 0):
      proposal_flag = 1
      proposal_centroid = fvector
    else:
      diff = np.subtract(fvector, proposal_centroid)
      mult = np.multiply(diff, learning_rate)
      proposal_centroid = np.add(mult, proposal_centroid)
    if(a_flag == 1):
      diff = np.subtract(fvector, affirmative_centroid)
      mult = np.multiply(diff, learning_rate)
      affirmative_centroid = np.subtract(affirmative_centroid, mult)
    if(f_flag == 1):
      diff = np.subtract(fvector, followup_centroid)
      mult = np.multiply(diff, learning_rate)
      followup_centroid = np.subtract(followup_centroid, mult)
    if(i_flag == 1):
      diff = np.subtract(fvector, invite_centroid)
      mult = np.multiply(diff, learning_rate)
      invite_centroid = np.subtract(invite_centroid, mult)

  elif(label_value==2):
    if(f_flag == 0):
      followup_flag = 1
      followup_centroid = fvector
    else:
      diff = np.subtract(fvector, followup_centroid)
      mult = np.multiply(diff, learning_rate)
      followup_centroid = np.add(mult, followup_centroid)
    if(a_flag == 1):
      diff = np.subtract(fvector, affirmative_centroid)
      mult = np.multiply(diff, learning_rate)
      affirmative_centroid = np.subtract(affirmative_centroid, mult)
    if(p_flag == 1):
      diff = np.subtract(fvector, proposal_centroid)
      mult = np.multiply(diff, learning_rate)
      proposal_centroid = np.subtract(proposal_centroid, mult)
    if(i_flag == 1):
      diff = np.subtract(fvector, invite_centroid)
      mult = np.multiply(diff, learning_rate)
      invite_centroid = np.subtract(invite_centroid, mult)

  elif(label_value==3):
    if(i_flag == 0):
      invite_flag = 1
      invite_centroid = fvector
    else:
      diff = np.subtract(fvector, invite_centroid)
      mult = np.multiply(diff, learning_rate)
      invite_centroid = np.add(mult, invite_centroid)
    if(a_flag == 1):
      diff = np.subtract(fvector, affirmative_centroid)
      mult = np.multiply(diff, learning_rate)
      affirmative_centroid = np.subtract(affirmative_centroid, mult)
    if(p_flag == 1):
      diff = np.subtract(fvector, proposal_centroid)
      mult = np.multiply(diff, learning_rate)
      proposal_centroid = np.subtract(proposal_centroid, mult)
    if(f_flag == 1):
      diff = np.subtract(fvector, followup_centroid)
      mult = np.multiply(diff, learning_rate)
      followup_centroid = np.subtract(followup_centroid, mult)

  #if((f_flag == 0) and (label_value==0)):
    #formal_flag = 1
    #formal_centroid = fvector
  #elif((if_flag == 0) and (label_value==1)):
    #informal_flag = 1
    #informal_centroid = fvector
    #print(fvector)
    #print(informal_centroid)
  #elif((f_flag ==1) and (label_value==0)):
    #diff = np.subtract(fvector, formal_centroid)
    #mult = np.multiply(diff, learning_rate)
    #formal_centroid = np.add(mult,formal_centroid)
  #elif((if_flag ==1) and (label_value==1)):
    #diff = np.subtract(fvector, informal_centroid)
    #mult = np.multiply(diff, learning_rate)
    #informal_centroid = np.add(mult,informal_centroid)
  else:
    print("Input Error")

  print("Updated Context Centroids!")
  #print("Affirmative:" + str(affirmative_centroid))
  #print("Proposal:" + str(proposal_centroid))
  #print("Followup:" + str(followup_centroid))
  #print("Invite:" + str(invite_centroid))

In [0]:
def cosineSim(a, b):
  cos_sim = dot(a, b)/(norm(a)*norm(b))
  return cos_sim

In [0]:
def generateIndex(max_val):
  affirmatives = 0
  proposals = 0
  followups = 0
  invites = 0
  for j in annotations:
    if j == 0:
      affirmatives += 1
    elif j == 1:
      proposals += 1
    elif j == 2:
      followups += 1
    elif j == 3:
      invites += 1
    else:
      print("Annotation Error, kindly check code and restart.. Annotate carefully!")
  cc_array = [affirmatives, proposals, followups, invites]
  centroids = [affirmative_centroid, proposal_centroid, followup_centroid, invite_centroid]

  #Check if any two elements in the list are equal
  seen = []
  repeat_flag = 0
  for each_number in cc_array:
    if each_number in seen:
      repeat_flag = 1
    else:
      seen.append(each_number)

  #Generate Random Index if elements are repeated or any element is 0
  if ((0 in cc_array) or (repeat_flag==1)):
    genIndex = random.randrange(max_val)
  #Or generate an index that is most similar to the least featured centroid
  else:
    #generating 10 random indices and storing their feature vectors
    vector_indices = []
    for i in range(10):
      vector_indices.append(random.randrange(max_val))
    index_features = [feature_vectors[x] for x in vector_indices]
    #finding least annotated centroid
    minInd = cc_array.index(min(cc_array))
    rarest_centroid = centroids[minInd]
    cos_sims = [cosineSim(y,rarest_centroid) for y in index_features]
    #print(cos_sims)
    #choosing feature index with highest similarity with the least featured centroid
    genInd = cos_sims.index(max(cos_sims))
    genIndex = vector_indices[genInd]
  return genIndex

In [0]:
def annotateLabels(num_labels):
  for i in range(num_labels):
    anno_index = generateIndex(3100)
    annotation_indices.append(anno_index)
    label_value = getLabel(anno_index)
    annotations.append(label_value)
    centroid_update(anno_index, label_value)

In [8]:
annotateLabels(50)

Message Contents: 
Subject: re : lobo payout
because the payback was done for october 2001 , we can ' t do it in pops so we will have to make manual entries to clear the imbalance from the books and show the sale from hpl to ena .
- - - - - original message - - - - -
from : hbcamp @ aep . com @ enron
sent : thursday , november 08 , 2001 5 : 15 pm
to : wynne , rita
cc : hall , bob m ; bcfisher @ aep . com ; rmbrownl @ aep . com ; daren . farmer @ enron . com
subject : re : lobo payout
rita ,
what time frame do intend to do this in ? i believe this contract is no longer effective after may 31 , so if it is in october you will have to reactivate the transport contract so you can book it in pops .
hc
- - - - - - - - - - - - - - - - - - - - - - forwarded by howard b camp / hol / aepin on 11 / 08 / 2001 05 : 12 pm - - - - - - - - - - - - - - - - - - - - - - - - - - -
to : " hbcamp @ aep . com @ enron "
cc : " hall , bob m " , , , , , ,
subject : re : lobo payout
i talked to daren and he ' s 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


Message Contents: 
Subject: fun pics . htm
-
- fun pics . htm

 Is the context of the displayed displayed message affirmative[0] or proposal[1] or followup[2] or invite[3] ?
Enter your value: 0
Updated Context Centroids!
Message Contents: 
Subject: kinder morgan ( cannon ) sale change
fyi - - -
please note that 7 , 305 of the kinder morgan sale was moved from hardy ( meter
68 ) to flagstaff ( meter 1606 ) intraday on the 4 th , and carried forward
through the end of the month due to pressure problems . this change was
confirmed with patsy shimek @ kinder morgan .
thanks ,
mark
x 33396

 Is the context of the displayed displayed message affirmative[0] or proposal[1] or followup[2] or invite[3] ?
Enter your value: 0
Updated Context Centroids!


In [11]:
print("Affirmative:" + str(affirmative_centroid))
print("Proposal:" + str(proposal_centroid))
print("Followup:" + str(followup_centroid))
print("Invite:" + str(invite_centroid))


Affirmative:[ 5.97993092e-01  1.22103271e+00 -3.70954264e-01 -4.10372411e-01
  9.81209238e-01  2.64300695e-01  1.16353741e+00 -3.86968556e-01
  9.41359788e-01  1.12464837e+00  1.28538816e+00 -1.01628159e+00
 -2.10630258e-01  1.31451790e+00 -4.25780421e-01 -2.25276418e-01
 -2.30111629e-01  8.64031460e-01  9.00727484e-01  1.43317323e-01
  1.41739267e-01  4.68081222e-01 -5.60879472e-01  6.97798342e-01
  6.73262235e-01  1.69992472e-02  4.05016831e-01  5.09352705e-02
 -9.24409226e-02 -2.05259732e-01 -2.40753090e-01  2.17597701e-01
  1.18524553e+00  1.18349189e-01  3.02815414e-01 -1.33434568e-02
 -4.10111775e-01 -3.99136715e-01  8.04537484e-02  2.62800350e-02
  0.00000000e+00  1.53651797e-01 -1.74418449e-01  5.38124812e-02
  3.71184246e-01 -2.03327427e-01  1.43531060e-01 -6.63161120e-02
  1.37842109e-01  1.49856274e+00  8.17726157e-01  9.84659450e-01
  3.85100408e-02  9.07768026e-01 -8.37234892e-02  3.49095380e-02
 -4.59019662e-01  1.18379699e+00  8.93621841e-01  1.81915844e-01
 -4.12048308e

In [12]:
print(annotation_indices)
print(annotations)

[910, 1509, 1704, 623, 2467, 504, 2684, 1893, 488, 2076, 2015, 976, 1814, 46, 2080, 1769, 2966, 159, 440, 2736, 410, 1861, 948, 2520, 1593, 2266, 2585, 217, 2394, 2605, 186, 77, 1623, 1041, 1996, 234, 1579, 2097, 1338, 1227, 1660, 677, 2939, 2654, 1493, 801, 2846, 2043, 2516, 1648]
[2, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 0, 2, 1, 3, 0, 2, 2, 0, 2, 2, 0, 1, 3, 1, 0, 0, 0]


In [13]:
print(len(annotation_indices))

50


In [0]:
#Storing annotation indices and annotations for testing
annoFile = 'drive/My Drive/Enron Spam/context_annotations.npy'
annoIndFile = 'drive/My Drive/Enron Spam/context_annotation_indices.npy'
#Storing context centroid vectors
affirmativeFile = 'drive/My Drive/Enron Spam/affirmative_vector.npy'
proposalFile = 'drive/My Drive/Enron Spam/proposal_vector.npy'
followupFile = 'drive/My Drive/Enron Spam/followup_vector.npy'
inviteFile = 'drive/My Drive/Enron Spam/invite_vector.npy'

np.save(annoFile, annotations)
np.save(annoIndFile, annotation_indices)

np.save(affirmativeFile, affirmative_centroid)
np.save(proposalFile, proposal_centroid)
np.save(followupFile, followup_centroid)
np.save(inviteFile, invite_centroid)